In [2]:
"test"

'test'

In [3]:
import cartopy
import cartopy.crs as ccrs # for geographic plotting
import cartopy.feature as cfeature
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xclim as xc
import xarray as xr
from cartopy.io.img_tiles import GoogleTiles
import os

ModuleNotFoundError: No module named 'cartopy'